In [8]:
from chemdataextractor import Document
from chemdataextractor.model import Compound
from chemdataextractor.doc import Paragraph, Heading
from chemdataextractor.parse.cem import CompoundParser

In [9]:
d = Document(
    Heading(u'DLSCORE: A Deep Learning algorithm to identify ligands'),
    Paragraph(u'The algorithm identified some of the ligands and their values, such as resorcinol with a logkp 3.79.'))


In [10]:
d.records.serialize()

[{'names': ['resorcinol']}]

In [11]:
from chemdataextractor.model import BaseModel, StringType, ListType, ModelType

class logkp(BaseModel):
    value = StringType()
    units = StringType()
    
Compound.logkp_points = ListType(ModelType(logkp))

In [47]:
import re
from chemdataextractor.parse import R, I, W, Optional, merge

prefix = (R(u'^logkp$', re.I) + Optional(I('of') | I('is')|I('was')| I('equals')|I('range')|I('=')|I('equivalent')
                                        |I('within')|I('varies')|I('were')|I('between')|I('values')|I('value')
                                        |I('number')).hide()
                                + Optional(I('of')|I('is')|I('to')|I('was')|I('were')|I('between')|I('equivalent'))
                                + Optional(I('to'))
        )
#units = Optional(R(u'^\.?$'))(u'units').add_action(merge)
#value = R(u'^\d+(\.\d+)?$')(u'value')
value = R(u'^(\-)?\d+(\.\d+)?$')(u'value')
logkp_model = (prefix + value)(u'logkp')

In [48]:
from chemdataextractor.parse.base import BaseParser
from chemdataextractor.utils import first

class logkpParser(BaseParser):
    root = logkp_model

    def interpret(self, result, start, end):
        compound = Compound(
            logkp_points=[
                logkp(
                    value=first(result.xpath('./value/text()'))
                    #units=first(result.xpath('./units/text()'))
                )
            ]
        )
        cem_el = first(result.xpath('/Users/dshrestha/Downloads/ChemDataExtractor-master/chemdataextractor/parse/cem'))
        if cem_el is not None:
            compound.names = cem_el.xpath('/Users/dshrestha/Downloads/ChemDataExtractor-master/chemdataextractor/parse/name/text()')
            compound.labels = cem_el.xpath('/Users/dshrestha/Downloads/ChemDataExtractor-master/chemdataextractor/parse/label/text()')
        yield compound


In [50]:
Paragraph.parsers = [CompoundParser(),logkpParser()]

In [52]:
d = Document(
    Heading(u'DLSCORE: A Deep Learning algorithm to identify ligands'),
    Paragraph(u'The algorithm identified some of the ligands and their values, such as resorcinol with a logkp range was -555553.88 - 9.8'))
d.records.serialize()

[{'names': ['resorcinol']}, {'logkp_points': [{'value': '-555553.88'}]}]